# Build a full-stack webapp

### Simple Webserver

In [1]:
const handler = (req) => {
    // Create a new response object
    const body = new TextEncoder().encode("Hello World");
    return new Response(body, {
        status: 200
    })
}

In [3]:
let server = Deno.serve({
    port: 8001
}, handler)

Listening on http://localhost:8001/


In [4]:
const response = await fetch("http://localhost:8001")
await response.text()

"Hello World"

In [5]:
await server.shutdown()

Interrupted: operation canceled

## Setup the Docs, Index and Query Engine

In [7]:
import * as mod from "https://deno.land/std@0.213.0/dotenv/mod.ts";
import { 
    Document, 
    VectorStoreIndex, 
    SimpleDirectoryReader 
} from "npm:llamaindex@0.1.8"
const keys = await mod.load({export:true}) // read API key from .env

const documents = await new SimpleDirectoryReader().loadData({directoryPath: "../data"})
const index = await VectorStoreIndex.fromDocuments(documents)
const queryEngine = index.asQueryEngine()

### Create Req/Res Handlers

In [10]:
const handler2 = async ( req ) => {
    if(req.method == "POST") {
        let data = await req.json();
        let answer = await queryEngine.query({
            query: data.query
        });
        let responseObj = {
            response: answer.toString()
        }
        return new Response(JSON.stringify(responseObj), {
            status: 200
        });
    } else {
        return new Response("Not Found", {
            status: 404
        })
    }
}
let server2 = Deno.serve({
    port: 8002
}, handler2);

Listening on http://localhost:8002/


In [11]:
let data = { query: "Brief me what is there in the corpus in 100 words" }

In [12]:
let response2 = await fetch("http://localhost:8002", {
  method: "POST",
  headers: {
    "Content-Type": "application/json"
  },
  body: JSON.stringify(data) // Convert the JavaScript object to a JSON string
})

In [13]:
let responseObj = await response2.json()
console.log(responseObj.response)

The corpus consists of 12 rules or teachings shared by Shams, focusing on the relationship between individuals and God, the importance of following the heart over the mind, the significance of love in spiritual growth, the role of language in understanding, the journey within oneself, the necessity of hardship for personal growth, the transformative power of seeking love, and the distinction between true mentors and false gurus. Each rule emphasizes the importance of self-discovery, love, and authenticity in spiritual and personal development, guiding individuals towards a deeper understanding of themselves and their connection to the divine.


In [14]:
await server2.shutdown()